In [1]:
import numpy as np
import tensorflow as tf
import random as rn
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
rn.seed(1)
from keras import backend as K
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph())
K.set_session(sess)
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.layers import Conv1D,MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import RMSprop
import keras.regularizers
import scipy
import math
import sys
import pandas as pd
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics import mean_squared_error
from scipy.stats import linregress
from scipy import interpolate
from scipy import signal
import cPickle as pickle
from video_process_utils import *
import collections

Using TensorFlow backend.


In [3]:
target_col = "O2cons"

In [4]:
#assign train/validation/test ids
alldata_processed =\
    pd.read_csv("data/annotations/alldata_processed_with_dev_residual.csv" )
alldata_processed['videoid'] = alldata_processed['videoid'].apply(lambda x: int(x))
alldata_processed = alldata_processed[alldata_processed[target_col].notnull()]
alldata_processed = alldata_processed.groupby(['videoid','side']).head(1)
ids_nonmissing_target = set(alldata_processed['videoid'].unique())

In [6]:
datasplit_df = pd.read_csv('./data/train_test_valid_id_split.csv')
datasplit_df['videoid'] = datasplit_df['videoid'].apply(lambda x: int(x))
all_ids = set(datasplit_df['videoid']).intersection(ids_nonmissing_target)
train_ids = set(datasplit_df[datasplit_df['dataset'] == 'train']['videoid']).intersection(ids_nonmissing_target)
validation_ids = set(datasplit_df[datasplit_df['dataset'] == 'validation']['videoid']).intersection(ids_nonmissing_target)
test_ids = set(datasplit_df[datasplit_df['dataset'] == 'test']['videoid']).intersection(ids_nonmissing_target)

In [7]:
with open('./data/all_processed_video_segments.pickle', 'r') as handle:
    processed_video_segments = pickle.load(handle)

In [37]:
x_columns = [2*LANK,2*LANK+1,2*LKNE,2*LKNE+1,
        2*LHIP,2*LHIP+1,2*LBTO,2*LBTO+1,
                  2*RANK,2*RANK+1,2*RKNE,2*RKNE+1,
        2*RHIP,2*RHIP+1,2*RBTO,2*RBTO+1,50,51,52,53,54,55,56]

target_dict = {}
for i in range(len(alldata_processed)):
    row = alldata_processed.iloc[i]
    target_dict[row['videoid']] = row[target_col]

In [40]:
X = [t[2] for t in processed_video_segments if t[0] in all_ids]
X = np.stack(X)[:,:,x_columns]

In [39]:
X_train = [t[2] for t in processed_video_segments if t[0] in train_ids]
X_train = np.stack(X_train)[:,:,x_columns]

In [13]:
X_validation = [t[2] for t in processed_video_segments if t[0] in validation_ids]
X_validation = np.stack(X_validation)[:,:,x_columns]

In [41]:
y = np.array([target_dict[t[0]] for t in processed_video_segments if t[0] in all_ids])

In [15]:
y_train = np.array([target_dict[t[0]] for t in processed_video_segments if t[0] in train_ids])

In [16]:
y_validation = np.array([target_dict[t[0]] for t in processed_video_segments if t[0] in validation_ids])

In [19]:
videoid_count_dict = collections.Counter(np.array([t[0] for t in processed_video_segments]))

In [20]:
train_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in train_ids]
train_videoid_weights = np.array(train_videoid_weights).reshape(-1,1)
validation_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in validation_ids]
validation_videoid_weights = np.array(validation_videoid_weights).reshape(-1,1)

In [21]:
target_min = np.min(y_train,axis=0)
target_range = np.max(y_train,axis=0) - np.min(y_train,axis=0)

In [22]:
y_train_scaled = ((y_train-target_min)/target_range).reshape(-1,1)
y_validation_scaled = ((y_validation-target_min)/target_range).reshape(-1,1)

y_validation_scaled = np.hstack([y_validation_scaled,validation_videoid_weights])
y_train_scaled = np.hstack([y_train_scaled,train_videoid_weights])
c_i_factor = np.mean(np.vstack([train_videoid_weights,validation_videoid_weights]))

In [23]:
vid_length = 124

In [24]:
def step_decay(initial_lrate,epochs_drop,drop_factor):
    def step_decay_fcn(epoch):
        return initial_lrate * math.pow(drop_factor, math.floor((1+epoch)/epochs_drop))
    return step_decay_fcn

epochs_drop,drop_factor = (10,0.8)
initial_lrate = 0.001
dropout_amount = 0.5
last_layer_dim = 10
filter_length = 8
conv_dim = 32
l2_lambda = 10**(-3.5)

def w_mse(weights):
    def loss(y_true, y_pred):
        #multiply by len(weights) to make the magnitude invariant to number of components in target
        return K.mean(K.sum(K.square(y_true-y_pred)*weights,axis=1)*tf.reshape(y_true[:,-1],(-1,1)))/c_i_factor
    return loss

#we don't want to optimize for the column counting video occurences of course, but
#they are included in the target so we can use that column for the loss function
weights = [1.0,0]
normal_weights = [1.0,0]


#normalize weights to sum to 1 to prevent affecting loss function
weights = weights/np.sum(weights)
normal_weights = normal_weights/np.sum(normal_weights)

#fixed epoch budget of 100 that empirically seems to be sufficient 
n_epochs = 100

mse_opt = w_mse(weights)

#monitor our actual objective
mse_metric = w_mse(target_range**2*normal_weights)

hyper_str = "params_"
for param in [initial_lrate,epochs_drop,drop_factor,dropout_amount,conv_dim,last_layer_dim,filter_length,l2_lambda]:
    hyper_str = hyper_str + str(param) + "_"

K.clear_session()
#K.set_session(sess)

model = Sequential()
model.add(Conv1D(conv_dim,filter_length, input_dim=X_train.shape[2],input_length=vid_length,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(dropout_amount))
model.add(Flatten())
model.add(Dense(last_layer_dim,activation='relu'))
model.add(Dense(2, activation='linear'))

/home/bryan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 8, padding="same", input_shape=(124, 23))`


In [29]:
checkpoint_folder = "/home/bryan/clinical-video/cnn_checkpoints_%s" % (target_col)

In [54]:
from keras.callbacks import LearningRateScheduler

from keras.callbacks import ModelCheckpoint
from keras.callbacks import TerminateOnNaN

train_model = True

if not os.path.exists(checkpoint_folder):
    os.makedirs(checkpoint_folder)

filepath=checkpoint_folder+"/weights-{epoch:02d}-{val_loss_2:.4f}.hdf5"
if train_model:

    opt = RMSprop(lr=0.0,rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(loss=mse_opt,metrics=[mse_metric],
                  optimizer=opt)



    checkpoint = \
        ModelCheckpoint(filepath, monitor='val_loss_2', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    lr = LearningRateScheduler(step_decay(initial_lrate,epochs_drop,drop_factor))

    history = model.fit(X_train, y_train_scaled,callbacks=[checkpoint,lr,TerminateOnNaN()],
              validation_data=(X_validation,y_validation_scaled),
              batch_size=32, epochs=n_epochs,shuffle=True)

Train on 16519 samples, validate on 1915 samples
Epoch 1/100
16519/16519 [==============================] - 23s - loss_1: 0.1773 - loss_2: 0.0227 - val_loss_1: 0.1085 - val_loss_2: 0.0126
Epoch 2/100
16519/16519 [==============================] - 21s - loss_1: 0.0445 - loss_2: 0.0035 - val_loss_1: 0.0381 - val_loss_2: 0.0036
Epoch 3/100
16519/16519 [==============================] - 20s - loss_1: 0.0312 - loss_2: 0.0033 - val_loss_1: 0.0279 - val_loss_2: 0.0034
Epoch 4/100
16519/16519 [==============================] - 21s - loss_1: 0.0233 - loss_2: 0.0031 - val_loss_1: 0.0243 - val_loss_2: 0.0036
Epoch 5/100
16519/16519 [==============================] - 21s - loss_1: 0.0204 - loss_2: 0.0030 - val_loss_1: 0.0156 - val_loss_2: 0.0023
Epoch 6/100
16519/16519 [==============================] - 21s - loss_1: 0.0189 - loss_2: 0.0029 - val_loss_1: 0.0180 - val_loss_2: 0.0028
Epoch 7/100
16519/16519 [==============================] - 21s - loss_1: 0.0179 - loss_2: 0.0028 - val_loss_1: 0.0233

16519/16519 [==============================] - 21s - loss_1: 0.0120 - loss_2: 0.0019 - val_loss_1: 0.0126 - val_loss_2: 0.0020
Epoch 51/100
16519/16519 [==============================] - 22s - loss_1: 0.0119 - loss_2: 0.0019 - val_loss_1: 0.0132 - val_loss_2: 0.0021
Epoch 52/100
16519/16519 [==============================] - 22s - loss_1: 0.0119 - loss_2: 0.0019 - val_loss_1: 0.0134 - val_loss_2: 0.0021
Epoch 53/100
16519/16519 [==============================] - 22s - loss_1: 0.0119 - loss_2: 0.0019 - val_loss_1: 0.0141 - val_loss_2: 0.0022
Epoch 54/100
16519/16519 [==============================] - 21s - loss_1: 0.0118 - loss_2: 0.0019 - val_loss_1: 0.0130 - val_loss_2: 0.0021
Epoch 55/100
16519/16519 [==============================] - 22s - loss_1: 0.0119 - loss_2: 0.0019 - val_loss_1: 0.0147 - val_loss_2: 0.0023
Epoch 56/100
16519/16519 [==============================] - 22s - loss_1: 0.0117 - loss_2: 0.0018 - val_loss_1: 0.0134 - val_loss_2: 0.0021
Epoch 57/100
16519/16519 [=======

16519/16519 [==============================] - 22s - loss_1: 0.0110 - loss_2: 0.0017 - val_loss_1: 0.0136 - val_loss_2: 0.0022
Epoch 76/100
16519/16519 [==============================] - 22s - loss_1: 0.0110 - loss_2: 0.0017 - val_loss_1: 0.0139 - val_loss_2: 0.0022
Epoch 77/100
16519/16519 [==============================] - 22s - loss_1: 0.0111 - loss_2: 0.0018 - val_loss_1: 0.0141 - val_loss_2: 0.0023
Epoch 78/100
16519/16519 [==============================] - 22s - loss_1: 0.0110 - loss_2: 0.0017 - val_loss_1: 0.0138 - val_loss_2: 0.0022
Epoch 79/100
16519/16519 [==============================] - 22s - loss_1: 0.0110 - loss_2: 0.0017 - val_loss_1: 0.0134 - val_loss_2: 0.0021
Epoch 80/100
16519/16519 [==============================] - 22s - loss_1: 0.0108 - loss_2: 0.0017 - val_loss_1: 0.0137 - val_loss_2: 0.0022
Epoch 81/100
16519/16519 [==============================] - 22s - loss_1: 0.0107 - loss_2: 0.0017 - val_loss_1: 0.0138 - val_loss_2: 0.0022
Epoch 82/100
16519/16519 [=======

16519/16519 [==============================] - 22s - loss_1: 0.0103 - loss_2: 0.0016 - val_loss_1: 0.0137 - val_loss_2: 0.0022


In [30]:
import statsmodels.api as sm

In [31]:
def undo_scaling(y,target_range,target_min):
    return y*target_range+target_min

In [32]:
weight_files = os.listdir(checkpoint_folder)
weight_files_df = pd.DataFrame(weight_files,columns=['filename'])
weight_files_df['num'] = weight_files_df['filename'].apply(lambda x: int(x.split('-')[1]))
weight_files_df.sort_values(by='num',ascending=True,inplace=True)

In [33]:
def predict_and_aggregate_singlevar(y,X,ids,model,target_col):
    df = pd.DataFrame(y,columns=[target_col])
    target_col_pred = target_col + "_pred"
    videoids = [t[0] for t in processed_video_segments if t[0] in ids]
    df["videoid"] = np.array(videoids)
    preds = model.predict(X)
    df[target_col_pred] = undo_scaling(preds[:,0],target_range,target_min)
    df["count"] = 1
    df = df.groupby(['videoid'],as_index=False).agg({target_col_pred:np.mean,'count':np.sum,target_col:np.mean})
    df['ones'] = 1
    return df

In [42]:
video_ids = [t[0] for t in processed_video_segments if t[0] in all_ids]
predictions_df = pd.DataFrame(video_ids,columns=['videoid'])
predictions_df[target_col] = y
predictions_df = predictions_df.merge(right=datasplit_df[['videoid','dataset']],on=['videoid'],how='left')

In [43]:
for i in range(0,len(weight_files_df)):
    weight_file = weight_files_df['filename'].iloc[i]
    print(weight_file)
    model.load_weights(checkpoint_folder + "/%s" % (weight_file))
    preds = model.predict(X)
    predictions_df["%s_pred_%s" % (target_col,i)] = undo_scaling(preds[:,0],target_range,target_min)

weights-00-0.0126.hdf5
weights-01-0.0036.hdf5
weights-02-0.0034.hdf5
weights-03-0.0036.hdf5
weights-04-0.0023.hdf5
weights-05-0.0028.hdf5
weights-06-0.0036.hdf5
weights-07-0.0025.hdf5
weights-08-0.0027.hdf5
weights-09-0.0028.hdf5
weights-10-0.0021.hdf5
weights-11-0.0021.hdf5
weights-12-0.0022.hdf5
weights-13-0.0020.hdf5
weights-14-0.0020.hdf5
weights-15-0.0107.hdf5
weights-16-0.0025.hdf5
weights-17-0.0026.hdf5
weights-18-0.0027.hdf5
weights-19-0.0026.hdf5
weights-20-0.0025.hdf5
weights-21-0.0022.hdf5
weights-22-0.0024.hdf5
weights-23-0.0024.hdf5
weights-24-0.0021.hdf5
weights-25-0.0021.hdf5
weights-26-0.0027.hdf5
weights-27-0.0020.hdf5
weights-28-0.0021.hdf5
weights-29-0.0025.hdf5
weights-30-0.0022.hdf5
weights-31-0.0024.hdf5
weights-32-0.0021.hdf5
weights-33-0.0023.hdf5
weights-34-0.0020.hdf5
weights-35-0.0027.hdf5
weights-36-0.0021.hdf5
weights-37-0.0023.hdf5
weights-38-0.0024.hdf5
weights-39-0.0021.hdf5
weights-40-0.0023.hdf5
weights-41-0.0022.hdf5
weights-42-0.0022.hdf5
weights-43-

In [46]:
predictions_df.groupby(['videoid','dataset'],as_index=False).mean().to_csv("./predictions/cnn_%s_predictions_all_epochs.csv" % (target_col),index=False)